# 1) Impelementing roman2urdu()

### Via word mapping and using extra data from https://github.com/MoizRauf/Urdu--Roman-Urdu--English--Dictionary/blob/master/en_ur_rom.high.tsv

In [164]:
import pandas as pd
from nltk.tokenize import word_tokenize
import string

Reading file 1

In [165]:
defval = lambda: []
from collections import defaultdict

map_dict = defaultdict(defval)

df2 = pd.read_csv('urdu roman.tsv', sep='\t')
df2.drop(['Ur—Rom Confidence Score', 'English',	'Ur-En Confidence Score'], axis=1, inplace=True)

map_dict = defaultdict(defval)

for i in range(len(df2)):
    map_dict[df2['Roman-Urdu'][i]].append(df2['Urdu'][i])

Reading file 2

In [166]:
file = open('urdu roman.txt', 'r', encoding='utf-8')
Lines = file.readlines()

urdu = []
roman = []

#add even lines to urdu list, and odd lines to english list
for i in range(len(Lines)):
    if i % 2 == 0:
        roman.append(Lines[i])
    else:
        urdu.append(Lines[i])

for i in range(len(urdu)):
    urdu[i] = urdu[i].strip()
    roman[i] = roman[i].strip().lower()

df = pd.DataFrame({'urdu': urdu, 'roman': roman})

for i in range(len(df)):
    roman_words = word_tokenize(df['roman'][i].translate(str.maketrans('', '', string.punctuation)))
    urdu_words = word_tokenize(df['urdu'][i].translate(str.maketrans('', '', '،۔')))
    
    
    if len(roman_words) == len(urdu_words):
        j = 0
        for i in range(len(roman_words)):
            if len(roman_words[i]) >=4  and len(urdu_words[j]) <= 2:
                if j < len(urdu_words)-1:
                    map_dict[roman_words[i]].append(urdu_words[j] + ' ' + urdu_words[j+1])
                    j+=1 
                else:
                    map_dict[roman_words[i]].append(urdu_words[j])
            else:
                map_dict[roman_words[i]].append(urdu_words[j])
            j+=1
            if j == len(urdu_words):
                break

        

Main transliteration function

In [167]:
map_dict['.'] = ['۔']

def word_segmentation(word):
    for i in range(0, len(word)):
        word1 = word[:len(word)-i]
        word2 = word[len(word)-i:]
        if word2 in map_dict and word1 in map_dict:
            return [map_dict[word1][0], map_dict[word2][0]], [word1, word2]
    return None, None

def roman2urdu(roman):
    urdu = []
    words = word_tokenize(roman)
    
    for i in range(len(words)):
        if words[i].lower() in map_dict.keys():
            urdu.append(map_dict[words[i].lower()][0])
        else:
            separated  = word_segmentation(words[i].lower())
            if separated == None:
                urdu.append(words[i].lower())
            else:
                urdu.append(separated[0])
                urdu.append(separated[1])
    return ' '.join(urdu)

Checking the function

In [168]:
output = roman2urdu('Sri Lanka apna aakhree over khail rahee hai aur score 264 hai is tarah mere khayal meN yah ek acha score hai aur match meN dilchaspee paida ho rahee hai')
print(output)

سری لنکا اپنا آخری اوور کھیل رہے هے اور سکور 264 هے اسس طرح ميرے خیال میں یہ ایک اچھا سکور هے اور میچ میں دلچسپی پیدا حو رہے هے


# 2) Implementing urdu2roman()

### Via character mapping and rulesets

In [169]:
urdu_lang_dict = {'ا': ['a', 'aa', 'a'], 'ب': ['b'], 'پ': ['p'], 'ت': ['t'], 'ٹ': ['t'], 'ث': ['s'], 'ج': ['j'], 'چ': ['ch'], 'ح': ['h'], 'خ': ['kh'], 'د': ['d'], 'ڈ': ['dd'], 'ذ': ['z'], 'ر': ['r'], 'ڑ': ['rr'], 'ز': ['z'], 'ژ': ['zh'], 'س': ['s'], 'ش': ['sh'], 'ص': ['s'], 'ض': ['z'], 'ط': ['t'], 'ظ': ['z'], 'ع': ['a', "a'", 'o'], 'غ': ['gh'], 'ف': ['f'], 'ق': ['q'], 'ک': ['k'], 'گ': ['g'], 'ل': ['l'], 'م': ['m'], 'ن': ['n'], 'و': ['wo', 'o', 'o'], 'ہ': ['h', 'h', 'eh'], 'ھ': ['h'], 'ه' : ['h'], 'ی': ['y', 'ei', 'i'], 'ئ': ['i'], 'ي': ['e'], 'ے': ['ay'], 'آ': ['aa'], 'ں': ['n'], '؟': ['?'], '،': [','], '.': ['.'], '۔': ['.']}

no_As = ['ی', 'ے', 'آ', 'ا', 'و', 'ع', 'ہ', 'ں', 'ھ', 'ئ', 'ن']
next_skippers = ['ئی']

Main transilteration function and a function to get position of letter in a word.

In [170]:
def get_position(text, index):
    if index == 0:
        return 0
    elif index == len(text) - 1:
        return 2
    else:
        if text[index - 1] == ' ':
            return 0
        elif text[index + 1] == ' ' or text[index + 1] == '.' or text[index + 1] == '۔' or text[index + 1] == '،' or text[index + 1] == '"':
            return 2
        else:
           return 1


def urdu2roman(input_text):
    skip = False
    output_text = ""
    for i in range(len(input_text)):
        if not skip:
            char = input_text[i]
            if char in urdu_lang_dict:
                pos = get_position(input_text, i)
                if len(urdu_lang_dict[char]) > 1:
                    output_text += urdu_lang_dict[char][pos]
                else:
                    output_text += urdu_lang_dict[char][0]
                if i < len(input_text) - 1:
                    if input_text[i+1] not in no_As and input_text[i] not in no_As:
                        if pos == 1:
                            output_text += 'a'
                    if char+input_text[i+1] in next_skippers:
                        skip = True
                        continue
            else:
                output_text += char
        else:
            skip = False
    return output_text

Checking the function

In [171]:
output = urdu2roman('سری لنکا اپنا آخری اوور کھیل رہے هے اور سکور 264 هے اسس طرح ميرے خیال میں یہ ایک اچھا سکور هے اور میچ میں دلچسپی پیدا حو رہے هے')
print(output)

sri lnka apna aakhari aoor kheil rhay hay aor skor 264 hay asas trah mearay kheiaal mein yeh aeik achha skor hay aor meich mein dlachasapi peida ho rhay hay
